In [6]:
# Import necessary libraries
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# Load the dataset
data = "data/tested.csv"
df = pd.read_csv(data) 

# Display basic info
print("Dataset Info:")
print(df.info())
print("\nFirst 5 rows:")
print(df.head())

# Check for duplicates based on exact matches
print("\nExact duplicates:", df.duplicated().sum())

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Survived     418 non-null    int64  
 2   Pclass       418 non-null    int64  
 3   Name         418 non-null    object 
 4   Sex          418 non-null    object 
 5   Age          332 non-null    float64
 6   SibSp        418 non-null    int64  
 7   Parch        418 non-null    int64  
 8   Ticket       418 non-null    object 
 9   Fare         417 non-null    float64
 10  Cabin        91 non-null     object 
 11  Embarked     418 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 39.3+ KB
None

First 5 rows:
   PassengerId  Survived  Pclass  \
0          892         0       3   
1          893         1       3   
2          894         0       2   
3          895         0       3   
4          896         1      

In [7]:
# Handle missing values
df['Age'] = df['Age'].fillna(df['Age'].median())  # Impute Age with median
df['Embarked'] = df['Embarked'].fillna(df['Embarked'].mode()[0])  # Impute Embarked with mode
df['Cabin'] = df['Cabin'].fillna('Unknown')  # Impute Cabin with placeholder

# Standardize 'Name' column
df['Name'] = df['Name'].str.strip().str.lower()  # Remove extra spaces and convert to lowercase

# Check for missing values after cleaning
print("\nMissing values after cleaning:")
print(df.isnull().sum())


Missing values after cleaning:
PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           1
Cabin          0
Embarked       0
dtype: int64


In [8]:
# Function to find similar names
def find_duplicates(names, threshold=90):
    duplicates = []
    for i, name in enumerate(names):
        # Compare with all other names
        matches = process.extract(name, names[i+1:], scorer=fuzz.token_sort_ratio)
        # Filter matches above threshold
        for match, score in matches:
            if score >= threshold:
                # Find the index of the match in names[i+1:]
                index = names[i+1:].index(match)
                duplicates.append((i, index + i + 1, name, match, score))
    return duplicates

# Apply fuzzy matching to 'Name' column
names = df['Name'].tolist()
duplicates = find_duplicates(names, threshold=90)

# Display potential duplicates
print("\nPotential duplicates (index, name1, name2, similarity score):")
for dup in duplicates:
    idx1, idx2, name1, name2, score = dup
    print(f"Index {idx1} vs {idx2}: {name1} | {name2} | Score: {score}")

# Example: Inspect duplicate rows
if duplicates:
    duplicate_indices = set([dup[0] for dup in duplicates] + [dup[1] for dup in duplicates])
    print("\nDuplicate rows:")
    print(df.iloc[list(duplicate_indices)][['PassengerId', 'Name', 'Age', 'Ticket', 'Pclass']])


Potential duplicates (index, name1, name2, similarity score):


In [9]:
# Remove duplicates based on fuzzy matches
if duplicates:
    indices_to_drop = [dup[1] for dup in duplicates]  # Keep first occurrence
    df_cleaned = df.drop(indices_to_drop).reset_index(drop=True)
    print(f"\nRemoved {len(indices_to_drop)} duplicate rows.")
else:
    df_cleaned = df.copy()
    print("\nNo duplicates found.")

# Verify the cleaned dataset
print("\nCleaned dataset info:")
print(df_cleaned.info())


No duplicates found.

Cleaned dataset info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Survived     418 non-null    int64  
 2   Pclass       418 non-null    int64  
 3   Name         418 non-null    object 
 4   Sex          418 non-null    object 
 5   Age          418 non-null    float64
 6   SibSp        418 non-null    int64  
 7   Parch        418 non-null    int64  
 8   Ticket       418 non-null    object 
 9   Fare         417 non-null    float64
 10  Cabin        418 non-null    object 
 11  Embarked     418 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 39.3+ KB
None


In [10]:
# Validate: Check for remaining duplicates
print("\nExact duplicates in cleaned dataset:", df_cleaned.duplicated().sum())

# Export cleaned dataset
df_cleaned.to_csv('data/cleaned_tested.csv', index=False)
print("\nCleaned dataset saved as 'data/cleaned_tested.csv'.")


Exact duplicates in cleaned dataset: 0

Cleaned dataset saved as 'data/cleaned_tested.csv'.
